In [2]:
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
#from fake_useragent import UserAgent
import requests
from selenium import webdriver
from collections import Counter
from tqdm.notebook import tqdm
import time
import openpyxl

In [3]:
def url_split(url):
    main_,para_1 = url.split("?")
    para_2 = para_1.split("&")
    para_3 = [d.replace("=",":").split(":") for d in para_2]
    params_ = {}
    for key,value in para_3:
        params_[key] = value
    return main_,params_

In [4]:
def find_product_list(product_name):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
          "referer": "https://www.coupang.com/"}
    row = "https://www.coupang.com/np/search?component=&q={}&channel=auto".format(product_name)
    main_,para = url_split(row)
    html = requests.get(main_,params=para,headers=headers).text
    soup = BeautifulSoup(html,"html.parser")
    url_source = {}
    product_list = {}
    for tag in soup.select("#productList li"):
        url_a = "https://www.coupang.com/vp/products/{}?isAddedCart=".format(tag["data-product-id"])
        try:
            product_list[tag["data-product-id"]] = int(tag.find(class_="rating-total-count").text.replace("(","").replace(")",""))
            url_source[url_a] =  int(tag.find(class_="rating-total-count").text.replace("(","").replace(")",""))
        except:
            pass
    return product_list#,url_source

In [5]:
def make_input_list(dic_,max_):
    def extract_p(value,pa_size):
        loop = value//pa_size
        remain = value%pa_size
        return loop+2
    par_ = []
    for key,value in dic_.items():
        pa_ = [key,extract_p(value,max_),max_]
        par_.append(pa_)
    return par_

In [6]:
def extraction_(soup_):
    data_all = []
    for tag in soup_.select(".sdp-review__article__list"):
        info_ = tag.find("div",class_="sdp-review")
        try:
            date_ = tag.find(class_="sdp-review__article__list__info__product-info__reg-date").text
            id_ = tag.find("span").text
            item_ = tag.find(class_="sdp-review__article__list__info__product-info__name").text
            liked_ = tag.find(class_="sdp-review__article__list__help js_reviewArticleHelpfulContainer")["data-count"]
            score_ = tag.find(class_="sdp-review__article__list__info__product-info__star-orange js_reviewArticleRatingValue")["data-rating"]
            try:
                contents = tag.find(class_="sdp-review__article__list__review__content js_reviewArticleContent").text.strip()
            except:
                contents = "None"
            channel_ = "Coupang"
            all_ = [date_,id_,item_,score_,contents,liked_,channel_]
            data_all.append(all_)
        except:
            pass
    df_ = pd.DataFrame(data_all).rename(columns={0:"date",1:"id",2:"item",3:"score",4:"contents",5:"liked",6:"channel"})
    return df_

In [7]:
def crawler_coupang(p_id,pages,pa_size):
    main_ = 'https://www.coupang.com/vp/product/reviews'
    df_all = pd.DataFrame()
    tq = tqdm(range(1,pages+2))
    for page in tq:
        params = {'productId': p_id,
         'page': str(page),
         'size': str(pa_size),
         'sortBy': 'DATE_DESC',
         'ratings': '',
         'q': '',
         'viRoleCode': '3',
         'ratingSummary': 'true'}
        header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
          "referer": "https://www.coupang.com/"}
        html = requests.get(main_,params=params,headers=header)
        while html.status_code != 200:
            html = requests.get(main_,params=params,headers=header)
            print("requests erorr")
        soup= BeautifulSoup(html.text,"html.parser")
        if len(extraction_(soup)) != 0:
            df_all = pd.concat([df_all,extraction_(soup)])
        else:
            print("Page Error :",page)
    df_all.drop_duplicates(subset=["date","id","item","score","contents","liked","channel"],keep = "first", inplace = True) 
    return df_all.reset_index(drop=True)

In [8]:
def coupan_all(input_s,path_):
    for input_ in input_s:
        ID = input_[0]
        pas = input_[1]
        pmax = input_[2]
        df_ = crawler_coupang(ID,pas,pmax)
        name_ = df_["item"].mode()[0].replace(" ","_").replace(".","").replace("/","_").replace("[","").replace("]","").replace("*","")
        print(name_)
        df_.to_excel(path_.format(name_))

In [9]:
pro_dic = find_product_list("헤어드라이기")
input_ = make_input_list(pro_dic,100)
path_ = r"C:\Users\Samsung\jupyter\Project_LG_textmining\data\hair\shopping_2\coupang\{}.xlsx"
coupan_all(input_,path_)

  0%|          | 0/4 [00:00<?, ?it/s]

Page Error : 3
Page Error : 4
플라이코_헤어드라이기_2000W,_FH6232KR,_화이트


  0%|          | 0/33 [00:00<?, ?it/s]

Page Error : 31
Page Error : 32
Page Error : 33
홈플래닛_전문가용_헤어_드라이기_RCY-119_2200W,_블랙


  0%|          | 0/329 [00:00<?, ?it/s]

...

Page Error : 4
Page Error : 5
필립스_듀얼_에어케어_헤어드라이기_2100W,_BHD340_19,_퍼플


  0%|          | 0/5 [00:00<?, ?it/s]

Page Error : 4
Page Error : 5
CKI_가정용_접이식_헤어_드라이어_CKI-D213_1200W,_화이트


  0%|          | 0/5 [00:00<?, ?it/s]

Page Error : 4
Page Error : 5
유닉스_BLDC_SUPER_D_플러스_럭셔리골드_에디션_헤어드라이기_UN-B3011_1800W,_혼합색상
